In [1]:
# Renaming to `BatchInteractiveProcessing_*.ipynb` from `BatchGenerateOutputs_*.ipynb` because it makes more sense
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
# %gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalLapsHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses, RankOrderResult, ShuffleHelper, Zscorer, LongShortStatsTuple, DirectionalRankOrderLikelihoods, DirectionalRankOrderResult, RankOrderComputationsContainer

# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalMergedDecodersResult

# BATCH_DATE_TO_USE = '2023-09-04' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2023-09-04_Apogee' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2024-09-12_GL' # used for filenames throught the notebook

# collected_outputs_path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs').resolve() # Linux
# collected_outputs_path: Path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs').resolve() # GreatLakes
# collected_outputs_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs').resolve() # Apogee

known_scripts_output_paths = [Path(v).resolve() for v in ['/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/']]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

collected_outputs_path = scripts_output_path.joinpath('../collected_outputs').resolve()
collected_outputs_path.mkdir(exist_ok=True)
assert collected_outputs_path.exists()
print(f'collected_outputs_path: {collected_outputs_path}')

# OUTPUTS: BATCH_DATE_TO_USE, collected_outputs_path

Automatic pdb calling has been turned OFF
scripts_output_path: /nfs/turbo/umms-kdiba/Data/Output/gen_scripts
collected_outputs_path: /nfs/turbo/umms-kdiba/Data/Output/collected_outputs


In [2]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/MAX/Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/home/halechr/FastData')] # , Path(r'/Volumes/MoverNew/data', Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    # display(good_only_batch_progress_df)
    display(batch_progress_df)
    

Computing loaded session pickle file results : "/nfs/turbo/umms-kdiba/Data/global_batch_result_2024-09-12_GL.pkl"... done.
Failure loading /nfs/turbo/umms-kdiba/Data/global_batch_result_2024-09-12_GL.pkl.
creating new batch_run
Saving (file mode 'w+b') pickle file results : "/nfs/turbo/umms-kdiba/Data/global_batch_result_2024-09-12_GL.pkl"... saved pickle file


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-08 14:26:15,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-09 01:22:43,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-09 03:23:37,0,0,0,0,False,,,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-12 15:55:31,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,kdiba,pin01,one,11-19_13-55-7,kdiba_pin01_one_11-19_13-55-7,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.NOT_STARTED,None,2009-11-19 13:55:07,0,0,0,0,False,,,,False,False
64,kdiba,pin01,one,fet11-01_12-58-54,kdiba_pin01_one_fet11-01_12-58-54,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,SessionBatchProgress.NOT_STARTED,None,2009-11-01 12:58:54,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,True,True
65,kdiba,pin01,one,fet11-03_11-0-53,kdiba_pin01_one_fet11-03_11-0-53,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,SessionBatchProgress.NOT_STARTED,None,2009-11-03 11:00:53,0,0,0,0,False,,,,False,False
66,kdiba,pin01,one,fet11-03_20-28-3,kdiba_pin01_one_fet11-03_20-28-3,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,SessionBatchProgress.NOT_STARTED,None,2009-11-03 20:28:03,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,False,True


# Run Batch Executions/Computations

In [3]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # OK
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'), # OK
]

good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)
good_session_concrete_folders

included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    display(included_session_batch_progress_df)

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-08 14:26:15,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-09 01:22:43,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-12 15:55:31,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 16:40:19,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,kdiba,vvp01,two,2006-4-10_12-58-3,kdiba_vvp01_two_2006-4-10_12-58-3,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,SessionBatchProgress.NOT_STARTED,None,2006-04-10 12:58:03,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,True,True
52,kdiba,pin01,one,11-02_17-46-44,kdiba_pin01_one_11-02_17-46-44,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.NOT_STARTED,None,2009-11-02 17:46:44,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True
53,kdiba,pin01,one,11-02_19-28-0,kdiba_pin01_one_11-02_19-28-0,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.NOT_STARTED,None,2009-11-02 19:28:00,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True
54,kdiba,pin01,one,11-03_12-3-25,kdiba_pin01_one_11-03_12-3-25,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.NOT_STARTED,None,2009-11-03 12:03:25,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True


# Execute Batch

In [4]:
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import determine_computation_datetimes_completion_function, compute_and_export_marginals_dfs_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, reload_exported_kdiba_session_position_info_mat_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function, export_session_h5_file_completion_function


# ==================================================================================================================== #
# BEGIN USER COMPLETION FUNCTIONS                                                                                      #
# ==================================================================================================================== #
custom_user_completion_functions = []

# custom_user_completion_functions.append(compute_and_export_marginals_dfs_completion_function)
# custom_user_completion_functions.append(perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function)
# custom_user_completion_functions.append(compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function)
# custom_user_completion_functions.append(reload_exported_kdiba_session_position_info_mat_completion_function)
custom_user_completion_functions.append(compute_and_export_session_instantaneous_spike_rates_completion_function)
custom_user_completion_functions.append(export_session_h5_file_completion_function)

# ==================================================================================================================== #
# END USER COMPLETION FUNCTIONS                                                                                        #
# ==================================================================================================================== #

# %pdb on
# multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=2)

# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
included_session_contexts = included_session_contexts # ALL

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts

# # No recomputing at all:
result_handler = BatchSessionCompletionHandler(force_reload_all=False,
                                                session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=SavingOptions.NEVER), # , override_file=
                                                global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=SavingOptions.NEVER),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
                                                **multiprocessing_kwargs)

# No Reloading
# result_handler = BatchSessionCompletionHandler(force_reload_all=False,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
#                                                 force_recompute_override_computations_includelist=None, # ['split_to_directional_laps', 'rank_order_shuffle_analysis']
#                                                 **multiprocessing_kwargs)


# # Forced Reloading:
# result_handler = BatchSessionCompletionHandler(force_reload_all=True,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
#                                                 **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session


## Append the desired completion functions:
result_handler.completion_functions = custom_user_completion_functions
# result_handler.completion_functions.append(compute_and_export_marginals_dfs_completion_function) # determine_computation_datetimes_completion_function
# result_handler.completion_functions.append(determine_computation_datetimes_completion_function)


## Specific Setup for 2023-09-28 Changes to LxC/SxC "refinements"
# result_handler.extended_computations_include_includelist = ['pf_computation', 
#                                                             'pfdt_computation',
#                                                             'firing_rate_trends',
#                                                 'pf_dt_sequential_surprise',
#                                                 'ratemap_peaks_prominence2d',
#                                                 'position_decoding', 
#                                                 # # 'position_decoding_two_step',
#                                                 'extended_stats',
#                                                 'long_short_decoding_analyses',
#                                                 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', #, 'long_short_rate_remapping',
#                                                 'long_short_inst_spike_rate_groups',
#                                                 'long_short_endcap_analysis',
#                                                 # 'spike_burst_detection',
#                                                 'split_to_directional_laps',
#                                                 'merged_directional_placefields',
#                                                 'rank_order_shuffle_analysis',
#                                                 'directional_train_test_split',
#                                                 'directional_decoders_decode_continuous',
#                                                 'directional_decoders_evaluate_epochs',    
# 												'directional_decoders_epoch_heuristic_scoring',
#                                                 ]


basic_local_computations = ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'position_decoding']
 
# basic_local_computations = [] # set to empty so we don't overwrite these computations
# result_handler.extended_computations_include_includelist = ['split_to_directional_laps', 'rank_order_shuffle_analysis'] # OVERRIDE with specific computations to do

result_handler.extended_computations_include_includelist = ['firing_rate_trends','split_to_directional_laps', 'long_short_decoding_analyses'] # OVERRIDE with specific computations to do

# result_handler.enable_hdf5_output = True # output the HDF5 when done.
result_handler.enable_hdf5_output = False

## Execute with the custom arguments.
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=False, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': basic_local_computations,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

len(included_session_contexts): 16
Beginning processing with len(included_session_contexts): 16
build_logger(full_logger_string="2024-09-12_09-09-46.gl3228.arc-ts.umich.edu.com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.kdiba.gor01.one.2006-6-08_14-26-15", file_logging_dir: None):
build_logger(full_logger_string="2024-09-12_09-09-46.gl3228.arc-ts.umich.edu.com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.kdiba.gor01.one.2006-6-09_1-22-43", file_logging_dir: None):
========================== runBatch STARTING ==========================
========================== runBatch STARTING ==========================	global_data_root_parent_path: /nfs/turbo/umms-kdiba/Data

	global_data_root_parent_path: /nfs/turbo/umms-kdiba/Data
	session_context: kdiba_gor01_one_2006-6-08_14-26-15
	session_context: kdiba_gor01_one_2006-6-09_1-22-43	session_basedir: "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15"

__________________________________________

saving_mode.shouldSave == False, so not saving at the end of batch_load_session
on_complete_success_execution_session(curr_session_context: kdiba_gor01_one_2006-6-08_14-26-15, curr_session_basedir: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15, ...)
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
were pipeline 

Computing loaded session pickle file results : "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl"... build_logger(full_logger_string="2024-09-12_09-09-08.gl3228.arc-ts.umich.edu.kdiba.gor01.one.2006-6-09_1-22-43", file_logging_dir: None):
done.
Loading pickled pipeline success: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the e

saving_mode.shouldSave == False, so not saving at the end of batch_load_session
on_complete_success_execution_session(curr_session_context: kdiba_gor01_one_2006-6-09_1-22-43, curr_session_basedir: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43, ...)
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
were pipeline pr

Computing loaded session pickle file results : "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl"... done.
_perform_long_short_instantaneous_spike_rate_groups_analysis is lacking a required computation config parameter! creating a new curr_active_pipeline.global_computation_results.computation_config
no changes in global results.
skipping figure generation because should_perform_figure_generation_to_file == False
	 time since last computation: 14 days, 19:34:20.028030
	 doing specific instantaneous firing rate computation for context: kdiba_gor01_one_2006-6-08_14-26-15...
setting LxC_aclus/SxC_aclus from user annotation.
DropShorterMode:
	minimum_event_duration present (minimum_event_duration=0.003).
	dropping 0 that are shorter than our minimum_event_duration of 0.003.	296 remain.
done.
_perform_long_short_instantaneous_spike_rate_groups_analysis is lacking a required computation config parameter! creating a new curr_active_pipeline.gl

...

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/MultiContextComputationFunctions/LongShortTrackComputations.py:284: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['firing_rates', 'is_neuron_active', 'active_aclus'], dtype='object')]

  self.rdf.rdf.to_hdf(file_path, key=f'{key}/rdf/df') # , format='table', data_columns=True Can't do 'table' format because `TypeError: Cannot serialize the column [firing_rates] because its data contents are not [string] but [mixed] object dtype`
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/MultiContextComputationFunctions/LongShortTrackComputations.py:290: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferre

long_short_inst_spike_rate_groups failed to save to HDF5 due to type issues and will be skipped. This is usually caused by Python None values. Error Object dtype dtype('O') has no native HDF5 equivalent
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields to .h5. Skipping.
done.
_perform_long_short_instantaneous_spike_rate_groups_analysis is lacking a required computation config parameter! creating a new curr_active_pipeline.global_computation_results.computation_config
no changes in global results.
skipping figure generation because should_perform_figure_generation_to_file == False
	 time since last computation: 2 days, 18:58:20.416792
	 doing specific instantaneous firing rate computation for context: kdiba_gor01_one_2006-6-07_11-26-53...
WARN: no user annotation for session_cell_exclusivity
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Wil

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-07_11-26-53'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/MultiContextComputationFunctions/LongShortTrackComputations.py:284: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['firing_rates', 'is_neuron_active', 'active_aclus'], dtype='object')]

  self.rdf.rdf.to_hdf(file_path, key=f'{key}/rdf/df') # , format='table', data_columns=True Can't do 'table' format because `TypeError: Cannot serialize the column [firing_rates] because its data

long_short_inst_spike_rate_groups is missing and will be skipped. Error: 'long_short_inst_spike_rate_groups'
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields to .h5. Skipping.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields to .h5. Skipping.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields to .h5. Skipping.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields to .h5. Skipping.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
could not output time-dependent placefields

MaybeEncodingError: Error sending result: '(<SessionBatchProgress.COMPLETED: 'COMPLETED'>, None, PipelineCompletionResult(long_epoch_name='maze1_any', long_laps=40 epochs
array([[   5.635867,   17.447765],
       [  31.862536,   39.770308],
       [ 135.801698,  144.175555],
       [ 161.458825,  167.33179 ],
       [ 234.465983,  239.806649],
       [ 255.120847,  262.696165],
       [ 294.026207,  299.799961],
       [ 314.047433,  319.385412],
       [ 499.299262,  504.805906],
       [ 511.57919 ,  518.352804],
       [ 530.199255,  540.640847],
       [ 558.3602  ,  565.501454],
       [ 584.353006,  591.260298],
       [ 599.268011,  604.739941],
       [ 616.818999,  625.727689],
       [ 645.746663,  655.190166],
       [ 678.313855,  684.687763],
       [ 692.428327,  697.566315],
       [ 712.4816  ,  721.724729],
       [ 734.337072,  741.178163],
       [ 750.48777 ,  755.223743],
       [ 763.13309 ,  768.97135 ],
       [ 782.252437,  788.22372 ],
       [ 804.741904,  81

In [5]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

Saving (file mode 'w+b') pickle file results : "/nfs/turbo/umms-kdiba/Data/global_batch_result_2024-09-12_GL.pkl"... 	moving new output at '/nfs/turbo/umms-kdiba/Data/20240912100131-global_batch_result_2024-09-12_GL.pkltmp' -> to desired location: '/nfs/turbo/umms-kdiba/Data/global_batch_result_2024-09-12_GL.pkl'
saved pickle file


In [6]:
# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
global_batch_run.to_hdf(file_path,'/')

done outputting HDF file.


# 2024-01-18 - Extracts the callback results 'determine_computation_datetimes_completion_function':


In [7]:
extracted_callback_fn_results = {a_sess_ctxt:a_result.across_session_results.get('determine_session_t_delta_completion_function', {}) for a_sess_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}
extracted_callback_fn_results

# {'session': list(extracted_callback_fn_results.keys()),
# Saves them to dataframe:
t_split_df = pd.DataFrame(extracted_callback_fn_results).T # .T to get 't_start', 't_delta', 't_end' as columns instead of rows and the index as the session name
t_split_df

t_split_csv_path = collected_outputs_path.joinpath(f'{BATCH_DATE_TO_USE}_t_split_df.csv').resolve()
print(f't_split_csv_path: {t_split_csv_path}')
t_split_df.to_csv(t_split_csv_path)
# t_split_df.to_csv(f'output/across_session_results/{BATCH_DATE_TO_USE}_t_split_df.csv')
# t_split_df.to_clipboard()

t_split_csv_path: /nfs/turbo/umms-kdiba/Data/Output/collected_outputs/2024-09-12_GL_t_split_df.csv


In [8]:
# Recover across_session_results_extended_dict
result_handler.across_sessions_instantaneous_fr_dict

{}

In [ ]:
result_handler.

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df

In [ ]:
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_files_metadata

# # Stores no global times
# global_pickle_path = curr_active_pipeline.global_computation_results_pickle_path.resolve()
# assert global_pickle_path.exists()
# global_pickle_path

# file_metadata = FilesystemMetadata.get_file_metadata(global_pickle_path)
# file_metadata['modification_time']

collected_global_computations_pickle_paths: List[Path] = good_only_batch_progress_df['global_computation_result_file'].to_list()

get_files_metadata(collected_global_computations_pickle_paths)



# Across Sessions After Batching Complete

In [ ]:
a_batch_progress_df = included_session_batch_progress_df.copy()

good_session_concrete_folders = [ConcreteSessionFolder(a_context, a_basedir) for a_context, a_basedir in zip(list(a_batch_progress_df.context.values), list(a_batch_progress_df.basedirs.values))]

# good_only_batch_progress_df.batch_results
# included_h5_paths = [get_file_str_if_file_exists(v.joinpath('output','pipeline_results.h5').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]
# included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]
included_h5_paths

In [ ]:
# target_dir = Path('output/across_session_results/2023-09-29').resolve()
# target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-09-29').resolve()
# target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-10-03').resolve()
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=target_dir)
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix='2023-10-05', only_include_file_types=['local_pkl', 'global_pkl','h5'])
copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['local_pkl', 'global_pkl','h5'])
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix='2023-10-07', only_include_file_types=['local_pkl', 'global_pkl','h5'])
copy_dict

In [ ]:
moved_files_dict_h5_files = copy_movedict(copy_dict)
moved_files_dict_h5_files

In [ ]:
moved_files_copydict_output_filename=f'backed_up_files_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = Path(global_data_root_parent_path).joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: {moved_files_copydict_file_path}')

## This just backs up all the pickles:
_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

In [ ]:
read_moved_files_dict_files = read_copydict_from_text_file(moved_files_copydict_file_path, debug_print=False)
read_moved_files_dict_files

In [ ]:
# read_moved_files_dict_files
restore_moved_files_dict_files = invert_filedict(read_moved_files_dict_files)
restore_moved_files_dict_files

In [ ]:
check_output_h5_files(included_h5_paths)

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

In [ ]:
## INPUTS: global_batch_run, 

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation
across_sessions_recomputed_instantaneous_fr_dict = {}

# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
        a_good_recomp_result = a_result.across_session_results.get('recomputed_inst_fr_comps', None)
        if a_good_recomp_result is not None:
            across_sessions_recomputed_instantaneous_fr_dict[a_ctxt] = a_good_recomp_result
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')


## Outputs: across_sessions_instantaneous_fr_dict, across_sessions_recomputed_instantaneous_fr_dict

In [ ]:
across_sessions_recomputed_instantaneous_fr_dict

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

across_session_result_long_short_recomputed_inst_firing_rate_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'

# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)

AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=collected_outputs_path.resolve(),
                                                 inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)


# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
## Export the non-recomputed version (unsure of the difference as of 2024-06-10):
across_session_result_long_short_inst_firing_rate_filename: str = f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'

# collected_outputs_path.joinpath(across_session_result_long_short_inst_firing_rate_filename).resolve()

# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=across_session_result_long_short_inst_firing_rate_filename) # '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2024-06-10_GL.pkl'

AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=collected_outputs_path.resolve(),
                                                 inst_fr_output_filename=across_session_result_long_short_inst_firing_rate_filename) 

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

# (neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'{BATCH_DATE_TO_USE}')
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=collected_outputs_path.resolve(), output_path_suffix=f'{BATCH_DATE_TO_USE}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)
# neuron_replay_stats_table['is_refined_LxC']

In [ ]:
long_short_fr_indicies_analysis_table

In [ ]:
neuron_replay_stats_table

In [ ]:
neuron_identities_table

In [ ]:
# np.sum(neuron_replay_stats_table['is_refined_LxC'])
# np.isnan(neuron_replay_stats_table['is_refined_LxC'])

In [ ]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
# _out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
# _out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership','is_refined_LxC', 'is_refined_SxC'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'is_refined_LxC', 'is_refined_SxC', 'count']
_out_value_counts_df

In [ ]:
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()

## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')

# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']

merged_df

In [ ]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
_out_value_counts_df

In [ ]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

In [ ]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
df.plot()


In [ ]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

In [ ]:
plt.close()

## Extract output files from all completed sessions:

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent

def save_filelist_to_text_file(hdf5_output_paths, filelist_path: Path):
    _out_string = '\n'.join([str(a_file) for a_file in hdf5_output_paths])
    print(f'{_out_string}')
    print(f'saving out to "{filelist_path}"...')
    with open(filelist_path, 'w') as f:
        f.write(_out_string)
    return _out_string, filelist_path

# Save output filelist:

# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}
session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)

# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from neuropy.core.epoch import Epoch

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
file_path
global_batch_run.to_hdf(file_path,'/')

In [ ]:
# # Get only the sessions with non-None results
# sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# # list(global_batch_run.session_batch_outputs.keys())

# # Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
# across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# # good_session_batch_outputs = global_batch_run.session_batch_outputs

# sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
# good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

# for a_ctxt, a_result in good_session_batch_outputs.items():
#     if a_result is not None:
#         # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
#         a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
#         if a_good_result is not None:
#             across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
#             # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
# num_sessions = len(across_sessions_instantaneous_fr_dict)
# print(f'num_sessions: {num_sessions}')

# # When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

# ## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl') # /nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2024-06-10_GL.pkl

# # ## Save pickle:
# # inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# # global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# # print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # # Save the all sessions instantaneous firing rate dict to the path:
# # saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
[a_ctxt.get_initialization_code_string() for a_ctxt in sessions_with_results]

# OLD

# 2023-10-06 - `joined_neruon_fri_df` loading

In [ ]:
# BATCH_DATE_TO_USE = '2023-10-05_NewParameters'
BATCH_DATE_TO_USE = '2023-10-07'
all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE)


In [ ]:

joined_neruon_fri_df_basename = f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df'
AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=joined_neruon_fri_df_basename, include_csv=False)
print(f'>>\t done with {output_file_prefix}')

# 2023-10-04 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'

inst_fr_output_load_filepath: Path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve() ## Load from existing:
assert inst_fr_output_load_filepath.exists()
inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
# output_path_suffix: str = f'2023-10-07'
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

In [ ]:
neuron_replay_stats_table

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table

In [ ]:
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='x_frs_index', title='Pf Peak position vs. LapsFRI', ylabel='Lap FRI')

long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='y_frs_index', title='Pf Peak position vs. ReplayFRI', ylabel='Replay FRI')

 #TODO 2023-10-05 11:40: - [ ] Extract the "contrarian cells", the ones that have a strong exclusivity on the laps but the opposite tendency on the replays


In [ ]:
# long_short_fr_indicies_analysis_table_filename = 'output/2023-10-07_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table_filename: str = f'output/{BATCH_DATE_TO_USE}_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table.to_csv(long_short_fr_indicies_analysis_table_filename)
print(f'saved: {long_short_fr_indicies_analysis_table_filename}')

# 2023-10-10 - Statistics for `across_sessions_bar_graphs`, analysing `across_session_inst_fr_computation` 

In [ ]:
import scipy.stats as stats
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_perform_diagonal_line_binomial_test, pho_stats_bar_graph_t_tests

binom_test_chance_result = pho_stats_perform_diagonal_line_binomial_test(long_short_fr_indicies_analysis_table)
print(f'binom_test_chance_result: {binom_test_chance_result}')

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)


## 2023-10-04 - Run `AcrossSessionsVisualizations` corresponding to the PhoDibaPaper2023 figures for all sessions


In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
ann_man = UserAnnotationsManager()
included_annotations = {ctxt:ann_man.annotations[ctxt].get('session_cell_exclusivity', None) for ctxt in included_session_contexts}

all_LxCs = []
all_SxCs = []

for ctxt, an_ann in included_annotations.items():
	session_ctxt_key:str = ctxt.get_description(separator='|', subset_includelist=IdentifyingContext._get_session_context_keys())
	all_LxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.LxC])
	all_SxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.SxC])
	
all_LxCs

In [ ]:
all_SxCs

In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo # used in post_compute_all_sessions_processing


## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

instantaneous_time_bin_size_seconds:float = 0.001

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
# _out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputLocation, ContextToPathMode, FileOutputManager # used in post_compute_all_sessions_processing
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo # used in post_compute_all_sessions_processing

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=True, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FileOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Replay_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=False, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
active_context = global_multi_session_context.adding_context_if_missing(run_date=BATCH_DATE_TO_USE)

_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=active_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)",
														defer_show=True, save_figure=True, enable_tiny_point_labels=False)

# _out_fig_2.perform_save()

# _out_fig_2.perform_save(global_multi_session_context, _fig_2_theta_out)




In [ ]:
debug_print: bool = False
enable_neptune: bool = True

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"

	
    # notebook_filepath

	# '/home/halechr/repos/Spike3D/BatchInteractiveProcessing_2024-06-11_GL.ipynb'

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        # neptuner.run = neptune.init_run(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath]) # see git_ref=GitRef(repository_path="/path/to/repo")
        # Add the session_context properties to the run: {'format_name': 'kdiba', 'animal': 'vvp01', 'exper_name': 'two', 'session_name': '2006-4-09_16-40-54'}

        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])

        # Create an AutoValueConvertingNeptuneRun and copy the attributes
        # neptuner.run = AutoValueConvertingNeptuneRun(base_run._client, base_run._uuid, base_run._url)
    
        params = {'BATCH_DATE_TO_USE': BATCH_DATE_TO_USE, "run_workstation": "GL"}
        neptuner.run["parameters"] = params

        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None

In [ ]:
for a_save_path, a_save_dict in registered_output_files.items():
	a_save_dict['fig']
	a_save_dict['context']
	neptuner.figures.upload(File(a_save_path.resolve().as_posix()))

In [ ]:

neptuner_run[f'output_files/across_sessions_fig_2'] = _out_fig_2

In [ ]:
if (neptuner is not None) and (neptuner_run is not None):
    neptuner.stop()